## The essential parts of the code comes from [this example notebook](https://drive.google.com/file/d/1XjFhd3eXx704tcdfQ4Q1OQn0LWKCRNJm/view)

In [1]:
import json, time, urllib.parse
import requests
import pandas as pd

df = pd.read_csv('thank_the_academy.AUG.2023b.csv.csv')

In [2]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include your email address which will allow them
# to contact you if something happens - such as - your code exceeding rate limits - or some other error 
REQUEST_HEADERS = {
    'User-Agent': '<hww1999@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = list(df['name'])

# This template is used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. The dictionary has a
# field/key for each of the required parameters. In the example, below, we only vary the article name, so the majority of the fields
# can stay constant for each request. Of course, these values *could* be changed if necessary.
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "",      # this should be changed for the different access types, which are desktop, mobile-app, mobile-web in this case
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015070100",   # start and end dates need to be set, e.g. 2015010100
    "end":         "2023100100"    # this is likely the wrong end date, e.g. 2023040100
}


In [3]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageviews_per_article(article_title = None, 
                                  access = None,
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

        
    # article title can be as a parameter to the call or in the request_template
    if access:
        request_template['access'] = access

    if not request_template['access']:
        raise Exception("Must supply an access type to make a pageviews request.")
        
    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'))
    request_template['article'] = article_title_encoded
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [4]:
# for website data from desktop
desktop = {"articles":[]}
for i in range(len(ARTICLE_TITLES)):
    print("Getting pageview data for: ",ARTICLE_TITLES[i])
    views = request_pageviews_per_article(ARTICLE_TITLES[i], 'desktop')
#     if 'items' in views.keys():
    try:
        print("Have %d months of pageview data"%(len(views['items'])))
        for month in views['items']:
            desktop["articles"].append(month)
    except Exception as e:
        print(e)
with open("academy_monthly_desktop_start201507-end202310.json", "a") as outfile:
        outfile.write(json.dumps(desktop,indent=4))

Getting pageview data for:  Everything Everywhere All at Once
Have 45 months of pageview data
Getting pageview data for:  All Quiet on the Western Front (2022 film)
Have 24 months of pageview data
Getting pageview data for:  The Whale (2022 film)
Have 15 months of pageview data
Getting pageview data for:  Top Gun: Maverick
Have 76 months of pageview data
Getting pageview data for:  Black Panther: Wakanda Forever
Have 29 months of pageview data
Getting pageview data for:  Avatar: The Way of Water
Have 59 months of pageview data
Getting pageview data for:  Women Talking (film)
Have 11 months of pageview data
Getting pageview data for:  Guillermo del Toro's Pinocchio
Have 21 months of pageview data
Getting pageview data for:  Navalny (film)
Have 21 months of pageview data
Getting pageview data for:  The Elephant Whisperers
Have 10 months of pageview data
Getting pageview data for:  An Irish Goodbye
Have 42 months of pageview data
Getting pageview data for:  The Boy, the Mole, the Fox and 

Have 93 months of pageview data
Getting pageview data for:  The White Helmets (film)
Have 81 months of pageview data
Getting pageview data for:  Zootopia
Have 99 months of pageview data
Getting pageview data for:  Spotlight (film)
Have 99 months of pageview data
Getting pageview data for:  Mad Max: Fury Road
Have 99 months of pageview data
Getting pageview data for:  The Revenant (2015 film)
Have 99 months of pageview data
Getting pageview data for:  Bridge of Spies (film)
Have 99 months of pageview data
Getting pageview data for:  The Big Short (film)
Have 99 months of pageview data
Getting pageview data for:  The Danish Girl (film)
Have 99 months of pageview data
Getting pageview data for:  Room (2015 film)
Have 99 months of pageview data
Getting pageview data for:  The Hateful Eight
Have 99 months of pageview data
Getting pageview data for:  Ex Machina (film)
Have 99 months of pageview data
Getting pageview data for:  Inside Out (2015 film)
Have 99 months of pageview data
Getting pa

Have 99 months of pageview data
Getting pageview data for:  Milk (2008 American film)
Have 40 months of pageview data
Getting pageview data for:  The Dark Knight (film)
Have 99 months of pageview data
Getting pageview data for:  WALL-E
Have 99 months of pageview data
Getting pageview data for:  The Reader (2008 film)
Have 99 months of pageview data
Getting pageview data for:  The Duchess (film)
Have 99 months of pageview data
Getting pageview data for:  Departures (2008 film)
Have 98 months of pageview data
Getting pageview data for:  Vicky Cristina Barcelona
Have 99 months of pageview data
Getting pageview data for:  Smile Pinki
Have 99 months of pageview data
Getting pageview data for:  Man on Wire
Have 99 months of pageview data
Getting pageview data for:  Toyland (film)
Have 99 months of pageview data
Getting pageview data for:  La Maison en Petits Cubes
Have 99 months of pageview data
Getting pageview data for:  No Country for Old Men (film)
Have 99 months of pageview data
Getting

Have 99 months of pageview data
Getting pageview data for:  Bowling for Columbine
Have 99 months of pageview data
Getting pageview data for:  8 Mile (film)
Have 99 months of pageview data
Getting pageview data for:  A Beautiful Mind (film)
Have 99 months of pageview data
Getting pageview data for:  The Lord of the Rings: The Fellowship of the Ring
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge!
Have 99 months of pageview data
Getting pageview data for:  Black Hawk Down (film)
Have 99 months of pageview data
Getting pageview data for:  Gosford Park
Have 99 months of pageview data
Getting pageview data for:  Monsters, Inc.
Have 99 months of pageview data
Getting pageview data for:  Pearl Harbor (film)
Have 99 months of pageview data
Getting pageview data for:  Iris (2001 film)
Have 99 months of pageview data
Getting pageview data for:  Shrek
Have 99 months of pageview data
Getting pageview data for:  Training Day
Have 99 months of pageview data
Getting pageview 

Have 99 months of pageview data
Getting pageview data for:  Speed (1994 film)
Have 99 months of pageview data
Getting pageview data for:  Ed Wood (film)
Have 99 months of pageview data
Getting pageview data for:  Pulp Fiction
Have 99 months of pageview data
Getting pageview data for:  Bullets Over Broadway
Have 99 months of pageview data
Getting pageview data for:  The Madness of King George
Have 99 months of pageview data
Getting pageview data for:  Legends of the Fall
Have 99 months of pageview data
Getting pageview data for:  A Time for Justice
Have 99 months of pageview data
Getting pageview data for:  Franz Kafka's It's a Wonderful Life
Have 99 months of pageview data
Getting pageview data for:  Maya Lin: A Strong Clear Vision
Have 99 months of pageview data
Getting pageview data for:  Burnt by the Sun
Have 99 months of pageview data
Getting pageview data for:  Trevor (film)
Have 99 months of pageview data
Getting pageview data for:  The Adventures of Priscilla, Queen of the Deser

...

Have 63 months of pageview data
Getting pageview data for:  Come Back, Little Sheba (1952 film)
Have 99 months of pageview data
Getting pageview data for:  Coquette (film)
Have 99 months of pageview data
Getting pageview data for:  The Country Cousin
Have 99 months of pageview data
Getting pageview data for:  The Country Girl (1954 film)
Have 99 months of pageview data
Getting pageview data for:  Cover Girl (film)
Have 99 months of pageview data
Getting pageview data for:  The Cowboy and the Lady (1938 film)
Have 99 months of pageview data
Getting pageview data for:  Crash Dive
Have 99 months of pageview data
Getting pageview data for:  Crashing the Water Barrier
Have 99 months of pageview data
Getting pageview data for:  The Critic (1963 film)
Have 15 months of pageview data
Getting pageview data for:  La Cucaracha (1934 film)
Have 99 months of pageview data
Getting pageview data for:  Cyrano de Bergerac (1950 film)
Have 99 months of pageview data
Getting pageview data for:  A Damsel 

Have 99 months of pageview data
Getting pageview data for:  The Heiress
Have 99 months of pageview data
Getting pageview data for:  Helen Keller in Her Story
Have 99 months of pageview data
Getting pageview data for:  Hello, Frisco, Hello
Have 99 months of pageview data
Getting pageview data for:  Henry V (1944 film)
Have 99 months of pageview data
Getting pageview data for:  Here Comes Mr. Jordan
Have 99 months of pageview data
Getting pageview data for:  Here Comes the Groom
Have 99 months of pageview data
Getting pageview data for:  The High and the Mighty (film)
Have 99 months of pageview data
Getting pageview data for:  High Noon
Have 99 months of pageview data
Getting pageview data for:  Hitler Lives
Have 99 months of pageview data
Getting pageview data for:  A Hole in the Head
Have 99 months of pageview data
Getting pageview data for:  The Hole (1962 film)
Have 99 months of pageview data
Getting pageview data for:  Holiday Inn (film)
Have 99 months of pageview data
Getting pagev

Have 99 months of pageview data
Getting pageview data for:  Moonbird
Have 99 months of pageview data
Getting pageview data for:  The More the Merrier
Have 99 months of pageview data
Getting pageview data for:  Morning Glory (1933 film)
Have 99 months of pageview data
Getting pageview data for:  Moscow Strikes Back
Have 99 months of pageview data
Getting pageview data for:  Mother Wore Tights
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge (1952 film)
Have 99 months of pageview data
Getting pageview data for:  Mouse Trouble
Have 99 months of pageview data
Getting pageview data for:  Mr. Deeds Goes to Town
Have 99 months of pageview data
Getting pageview data for:  Mr. Smith Goes to Washington
Have 99 months of pageview data
Getting pageview data for:  Mrs. Miniver
Have 99 months of pageview data
Getting pageview data for:  Munro (film)
Have 99 months of pageview data
Getting pageview data for:  The Music Box
Have 99 months of pageview data
Getting pageview data 

Have 99 months of pageview data
Getting pageview data for:  Shane (film)
Have 99 months of pageview data
Getting pageview data for:  Shanghai Express (film)
Have 99 months of pageview data
Getting pageview data for:  She Wore a Yellow Ribbon
Have 99 months of pageview data
Getting pageview data for:  Shoeshine (film)
Have 99 months of pageview data
Getting pageview data for:  The Silent World
Have 99 months of pageview data
Getting pageview data for:  The Sin of Madelon Claudet
Have 99 months of pageview data
Getting pageview data for:  Since You Went Away
Have 99 months of pageview data
Getting pageview data for:  Skippy (film)
Have 99 months of pageview data
Getting pageview data for:  Sky Above and Mud Beneath
Have 99 months of pageview data
Getting pageview data for:  The Snake Pit
Have 99 months of pageview data
Getting pageview data for:  Snow White and the Seven Dwarfs (1937 film)
Have 99 months of pageview data
Getting pageview data for:  So Much for So Little
Have 99 months of

Have 99 months of pageview data
Getting pageview data for:  The Vanishing Prairie
Have 99 months of pageview data
Getting pageview data for:  The Virgin Spring
Have 99 months of pageview data
Getting pageview data for:  Viva Villa!
Have 99 months of pageview data
Getting pageview data for:  Viva Zapata!
Have 99 months of pageview data
Getting pageview data for:  Waikiki Wedding
Have 99 months of pageview data
Getting pageview data for:  The Walls of Malapaga
Have 99 months of pageview data
Getting pageview data for:  The War of the Worlds (1953 film)
Have 99 months of pageview data
Getting pageview data for:  Watch on the Rhine
Have 99 months of pageview data
Getting pageview data for:  Water Birds
Have 99 months of pageview data
Getting pageview data for:  The Way of All Flesh (1927 film)
Have 97 months of pageview data
Getting pageview data for:  West Side Story (1961 film)
Have 62 months of pageview data
Getting pageview data for:  The Westerner (1940 film)
Have 52 months of pagevie

In [7]:
# for website data from mobile-app

mobile_app = {"articles":[]}
for i in range(len(ARTICLE_TITLES)):
    print("Getting pageview data for: ",ARTICLE_TITLES[i])
    views = request_pageviews_per_article(ARTICLE_TITLES[i], 'mobile-app')
    if 'items' in views.keys():
        print("Have %d months of pageview data"%(len(views['items'])))
        for month in views['items']:
            mobile_app["articles"].append(month)
with open("academy_monthly_mobile_app_start201507-end202310.json", "a") as outfile:
        outfile.write(json.dumps(mobile_app,indent=4))

Getting pageview data for:  Everything Everywhere All at Once
Have 45 months of pageview data
Getting pageview data for:  All Quiet on the Western Front (2022 film)
Have 24 months of pageview data
Getting pageview data for:  The Whale (2022 film)
Have 15 months of pageview data
Getting pageview data for:  Top Gun: Maverick
Have 76 months of pageview data
Getting pageview data for:  Black Panther: Wakanda Forever
Have 29 months of pageview data
Getting pageview data for:  Avatar: The Way of Water
Have 59 months of pageview data
Getting pageview data for:  Women Talking (film)
Have 11 months of pageview data
Getting pageview data for:  Guillermo del Toro's Pinocchio
Have 21 months of pageview data
Getting pageview data for:  Navalny (film)
Have 21 months of pageview data
Getting pageview data for:  The Elephant Whisperers
Have 10 months of pageview data
Getting pageview data for:  An Irish Goodbye
Have 42 months of pageview data
Getting pageview data for:  The Boy, the Mole, the Fox and 

Have 93 months of pageview data
Getting pageview data for:  The White Helmets (film)
Have 81 months of pageview data
Getting pageview data for:  Zootopia
Have 99 months of pageview data
Getting pageview data for:  Spotlight (film)
Have 99 months of pageview data
Getting pageview data for:  Mad Max: Fury Road
Have 99 months of pageview data
Getting pageview data for:  The Revenant (2015 film)
Have 99 months of pageview data
Getting pageview data for:  Bridge of Spies (film)
Have 99 months of pageview data
Getting pageview data for:  The Big Short (film)
Have 99 months of pageview data
Getting pageview data for:  The Danish Girl (film)
Have 99 months of pageview data
Getting pageview data for:  Room (2015 film)
Have 99 months of pageview data
Getting pageview data for:  The Hateful Eight
Have 99 months of pageview data
Getting pageview data for:  Ex Machina (film)
Have 99 months of pageview data
Getting pageview data for:  Inside Out (2015 film)
Have 99 months of pageview data
Getting pa

Have 99 months of pageview data
Getting pageview data for:  Milk (2008 American film)
Have 40 months of pageview data
Getting pageview data for:  The Dark Knight (film)
Have 99 months of pageview data
Getting pageview data for:  WALL-E
Have 99 months of pageview data
Getting pageview data for:  The Reader (2008 film)
Have 99 months of pageview data
Getting pageview data for:  The Duchess (film)
Have 99 months of pageview data
Getting pageview data for:  Departures (2008 film)
Have 98 months of pageview data
Getting pageview data for:  Vicky Cristina Barcelona
Have 99 months of pageview data
Getting pageview data for:  Smile Pinki
Have 99 months of pageview data
Getting pageview data for:  Man on Wire
Have 99 months of pageview data
Getting pageview data for:  Toyland (film)
Have 99 months of pageview data
Getting pageview data for:  La Maison en Petits Cubes
Have 99 months of pageview data
Getting pageview data for:  No Country for Old Men (film)
Have 99 months of pageview data
Getting

Have 99 months of pageview data
Getting pageview data for:  Twin Towers (film)
Have 99 months of pageview data
Getting pageview data for:  Bowling for Columbine
Have 99 months of pageview data
Getting pageview data for:  8 Mile (film)
Have 99 months of pageview data
Getting pageview data for:  A Beautiful Mind (film)
Have 99 months of pageview data
Getting pageview data for:  The Lord of the Rings: The Fellowship of the Ring
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge!
Have 99 months of pageview data
Getting pageview data for:  Black Hawk Down (film)
Have 99 months of pageview data
Getting pageview data for:  Gosford Park
Have 99 months of pageview data
Getting pageview data for:  Monsters, Inc.
Have 99 months of pageview data
Getting pageview data for:  Pearl Harbor (film)
Have 99 months of pageview data
Getting pageview data for:  Iris (2001 film)
Have 99 months of pageview data
Getting pageview data for:  Shrek
Have 99 months of pageview data
Getting pag

Have 99 months of pageview data
Getting pageview data for:  Toy Story
Have 99 months of pageview data
Getting pageview data for:  Forrest Gump
Have 99 months of pageview data
Getting pageview data for:  The Lion King
Have 99 months of pageview data
Getting pageview data for:  Speed (1994 film)
Have 99 months of pageview data
Getting pageview data for:  Ed Wood (film)
Have 99 months of pageview data
Getting pageview data for:  Pulp Fiction
Have 99 months of pageview data
Getting pageview data for:  Bullets Over Broadway
Have 99 months of pageview data
Getting pageview data for:  The Madness of King George
Have 99 months of pageview data
Getting pageview data for:  Legends of the Fall
Have 99 months of pageview data
Getting pageview data for:  A Time for Justice
Have 99 months of pageview data
Getting pageview data for:  Franz Kafka's It's a Wonderful Life
Have 99 months of pageview data
Getting pageview data for:  Maya Lin: A Strong Clear Vision
Have 99 months of pageview data
Getting p

...

Have 99 months of pageview data
Getting pageview data for:  City of Wax
Have 99 months of pageview data
Getting pageview data for:  Cleopatra (1934 film)
Have 99 months of pageview data
Getting pageview data for:  Cleopatra (1963 film)
Have 99 months of pageview data
Getting pageview data for:  Climbing the Matterhorn
Have 99 months of pageview data
Getting pageview data for:  Come and Get It (1936 film)
Have 63 months of pageview data
Getting pageview data for:  Come Back, Little Sheba (1952 film)
Have 99 months of pageview data
Getting pageview data for:  Coquette (film)
Have 99 months of pageview data
Getting pageview data for:  The Country Cousin
Have 99 months of pageview data
Getting pageview data for:  The Country Girl (1954 film)
Have 99 months of pageview data
Getting pageview data for:  Cover Girl (film)
Have 99 months of pageview data
Getting pageview data for:  The Cowboy and the Lady (1938 film)
Have 99 months of pageview data
Getting pageview data for:  Crash Dive
Have 99

Have 48 months of pageview data
Getting pageview data for:  The Guns of Navarone (film)
Have 99 months of pageview data
Getting pageview data for:  Hamlet (1948 film)
Have 99 months of pageview data
Getting pageview data for:  Heureux Anniversaire
Have 99 months of pageview data
Getting pageview data for:  Harvey (1950 film)
Have 44 months of pageview data
Getting pageview data for:  The Harvey Girls
Have 99 months of pageview data
Getting pageview data for:  Heavenly Music
Have 99 months of pageview data
Getting pageview data for:  The Heiress
Have 99 months of pageview data
Getting pageview data for:  Helen Keller in Her Story
Have 99 months of pageview data
Getting pageview data for:  Hello, Frisco, Hello
Have 99 months of pageview data
Getting pageview data for:  Henry V (1944 film)
Have 99 months of pageview data
Getting pageview data for:  Here Comes Mr. Jordan
Have 99 months of pageview data
Getting pageview data for:  Here Comes the Groom
Have 99 months of pageview data
Getting

Have 99 months of pageview data
Getting pageview data for:  The Milky Way (1940 film)
Have 99 months of pageview data
Getting pageview data for:  Min and Bill
Have 99 months of pageview data
Getting pageview data for:  Miracle on 34th Street
Have 99 months of pageview data
Getting pageview data for:  The Miracle Worker (1962 film)
Have 99 months of pageview data
Getting pageview data for:  Mister Roberts (1955 film)
Have 99 months of pageview data
Getting pageview data for:  Monsieur Vincent
Have 99 months of pageview data
Getting pageview data for:  Moonbird
Have 99 months of pageview data
Getting pageview data for:  The More the Merrier
Have 99 months of pageview data
Getting pageview data for:  Morning Glory (1933 film)
Have 99 months of pageview data
Getting pageview data for:  Moscow Strikes Back
Have 99 months of pageview data
Getting pageview data for:  Mother Wore Tights
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge (1952 film)
Have 99 months of pagev

Have 99 months of pageview data
Getting pageview data for:  Serengeti Shall Not Die
Have 99 months of pageview data
Getting pageview data for:  Sergeant York (film)
Have 99 months of pageview data
Getting pageview data for:  Seven Brides for Seven Brothers
Have 99 months of pageview data
Getting pageview data for:  Seven Days to Noon
Have 99 months of pageview data
Getting pageview data for:  The Seventh Veil
Have 99 months of pageview data
Getting pageview data for:  Shane (film)
Have 99 months of pageview data
Getting pageview data for:  Shanghai Express (film)
Have 99 months of pageview data
Getting pageview data for:  She Wore a Yellow Ribbon
Have 99 months of pageview data
Getting pageview data for:  Shoeshine (film)
Have 99 months of pageview data
Getting pageview data for:  The Silent World
Have 99 months of pageview data
Getting pageview data for:  The Sin of Madelon Claudet
Have 99 months of pageview data
Getting pageview data for:  Since You Went Away
Have 99 months of pagevi

Have 99 months of pageview data
Getting pageview data for:  Two Women
Have 99 months of pageview data
Getting pageview data for:  The Ugly Duckling (1939 film)
Have 99 months of pageview data
Getting pageview data for:  Underworld (1927 film)
Have 99 months of pageview data
Getting pageview data for:  The V.I.P.s (film)
Have 91 months of pageview data
Getting pageview data for:  Perfect Strangers (1945 film)
Have 99 months of pageview data
Getting pageview data for:  Van Gogh (1948 film)
Have 99 months of pageview data
Getting pageview data for:  The Vanishing Prairie
Have 99 months of pageview data
Getting pageview data for:  The Virgin Spring
Have 99 months of pageview data
Getting pageview data for:  Viva Villa!
Have 99 months of pageview data
Getting pageview data for:  Viva Zapata!
Have 99 months of pageview data
Getting pageview data for:  Waikiki Wedding
Have 99 months of pageview data
Getting pageview data for:  The Walls of Malapaga
Have 99 months of pageview data
Getting page

In [8]:
# for website data from mobile-web

mobile_web = {"articles":[]}
for i in range(len(ARTICLE_TITLES)):
    print("Getting pageview data for: ",ARTICLE_TITLES[i])
    views = request_pageviews_per_article(ARTICLE_TITLES[i], 'mobile-web')
    if 'items' in views.keys():
        print("Have %d months of pageview data"%(len(views['items'])))
        for month in views['items']:
            mobile_web["articles"].append(month)
with open("academy_monthly_mobile_web_start201507-end202310.json", "a") as outfile:
        outfile.write(json.dumps(mobile_web,indent=4))

Getting pageview data for:  Everything Everywhere All at Once
Have 45 months of pageview data
Getting pageview data for:  All Quiet on the Western Front (2022 film)
Have 24 months of pageview data
Getting pageview data for:  The Whale (2022 film)
Have 15 months of pageview data
Getting pageview data for:  Top Gun: Maverick
Have 76 months of pageview data
Getting pageview data for:  Black Panther: Wakanda Forever
Have 29 months of pageview data
Getting pageview data for:  Avatar: The Way of Water
Have 59 months of pageview data
Getting pageview data for:  Women Talking (film)
Have 11 months of pageview data
Getting pageview data for:  Guillermo del Toro's Pinocchio
Have 21 months of pageview data
Getting pageview data for:  Navalny (film)
Have 21 months of pageview data
Getting pageview data for:  The Elephant Whisperers
Have 10 months of pageview data
Getting pageview data for:  An Irish Goodbye
Have 42 months of pageview data
Getting pageview data for:  The Boy, the Mole, the Fox and 

Have 93 months of pageview data
Getting pageview data for:  The White Helmets (film)
Have 81 months of pageview data
Getting pageview data for:  Zootopia
Have 99 months of pageview data
Getting pageview data for:  Spotlight (film)
Have 99 months of pageview data
Getting pageview data for:  Mad Max: Fury Road
Have 99 months of pageview data
Getting pageview data for:  The Revenant (2015 film)
Have 99 months of pageview data
Getting pageview data for:  Bridge of Spies (film)
Have 99 months of pageview data
Getting pageview data for:  The Big Short (film)
Have 99 months of pageview data
Getting pageview data for:  The Danish Girl (film)
Have 99 months of pageview data
Getting pageview data for:  Room (2015 film)
Have 99 months of pageview data
Getting pageview data for:  The Hateful Eight
Have 99 months of pageview data
Getting pageview data for:  Ex Machina (film)
Have 99 months of pageview data
Getting pageview data for:  Inside Out (2015 film)
Have 99 months of pageview data
Getting pa

Have 99 months of pageview data
Getting pageview data for:  Milk (2008 American film)
Have 40 months of pageview data
Getting pageview data for:  The Dark Knight (film)
Have 99 months of pageview data
Getting pageview data for:  WALL-E
Have 99 months of pageview data
Getting pageview data for:  The Reader (2008 film)
Have 99 months of pageview data
Getting pageview data for:  The Duchess (film)
Have 99 months of pageview data
Getting pageview data for:  Departures (2008 film)
Have 98 months of pageview data
Getting pageview data for:  Vicky Cristina Barcelona
Have 99 months of pageview data
Getting pageview data for:  Smile Pinki
Have 99 months of pageview data
Getting pageview data for:  Man on Wire
Have 99 months of pageview data
Getting pageview data for:  Toyland (film)
Have 99 months of pageview data
Getting pageview data for:  La Maison en Petits Cubes
Have 99 months of pageview data
Getting pageview data for:  No Country for Old Men (film)
Have 99 months of pageview data
Getting

Have 99 months of pageview data
Getting pageview data for:  Bowling for Columbine
Have 99 months of pageview data
Getting pageview data for:  8 Mile (film)
Have 99 months of pageview data
Getting pageview data for:  A Beautiful Mind (film)
Have 99 months of pageview data
Getting pageview data for:  The Lord of the Rings: The Fellowship of the Ring
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge!
Have 99 months of pageview data
Getting pageview data for:  Black Hawk Down (film)
Have 99 months of pageview data
Getting pageview data for:  Gosford Park
Have 99 months of pageview data
Getting pageview data for:  Monsters, Inc.
Have 99 months of pageview data
Getting pageview data for:  Pearl Harbor (film)
Have 99 months of pageview data
Getting pageview data for:  Iris (2001 film)
Have 99 months of pageview data
Getting pageview data for:  Shrek
Have 99 months of pageview data
Getting pageview data for:  Training Day
Have 99 months of pageview data
Getting pageview 

Have 99 months of pageview data
Getting pageview data for:  The Lion King
Have 99 months of pageview data
Getting pageview data for:  Speed (1994 film)
Have 99 months of pageview data
Getting pageview data for:  Ed Wood (film)
Have 99 months of pageview data
Getting pageview data for:  Pulp Fiction
Have 99 months of pageview data
Getting pageview data for:  Bullets Over Broadway
Have 99 months of pageview data
Getting pageview data for:  The Madness of King George
Have 99 months of pageview data
Getting pageview data for:  Legends of the Fall
Have 99 months of pageview data
Getting pageview data for:  A Time for Justice
Have 99 months of pageview data
Getting pageview data for:  Franz Kafka's It's a Wonderful Life
Have 99 months of pageview data
Getting pageview data for:  Maya Lin: A Strong Clear Vision
Have 99 months of pageview data
Getting pageview data for:  Burnt by the Sun
Have 99 months of pageview data
Getting pageview data for:  Trevor (film)
Have 99 months of pageview data
G

...

Have 99 months of pageview data
Getting pageview data for:  Come and Get It (1936 film)
Have 63 months of pageview data
Getting pageview data for:  Come Back, Little Sheba (1952 film)
Have 99 months of pageview data
Getting pageview data for:  Coquette (film)
Have 99 months of pageview data
Getting pageview data for:  The Country Cousin
Have 99 months of pageview data
Getting pageview data for:  The Country Girl (1954 film)
Have 99 months of pageview data
Getting pageview data for:  Cover Girl (film)
Have 99 months of pageview data
Getting pageview data for:  The Cowboy and the Lady (1938 film)
Have 99 months of pageview data
Getting pageview data for:  Crash Dive
Have 99 months of pageview data
Getting pageview data for:  Crashing the Water Barrier
Have 99 months of pageview data
Getting pageview data for:  The Critic (1963 film)
Have 15 months of pageview data
Getting pageview data for:  La Cucaracha (1934 film)
Have 99 months of pageview data
Getting pageview data for:  Cyrano de Be

Have 44 months of pageview data
Getting pageview data for:  The Harvey Girls
Have 99 months of pageview data
Getting pageview data for:  Heavenly Music
Have 99 months of pageview data
Getting pageview data for:  The Heiress
Have 99 months of pageview data
Getting pageview data for:  Helen Keller in Her Story
Have 99 months of pageview data
Getting pageview data for:  Hello, Frisco, Hello
Have 99 months of pageview data
Getting pageview data for:  Henry V (1944 film)
Have 99 months of pageview data
Getting pageview data for:  Here Comes Mr. Jordan
Have 99 months of pageview data
Getting pageview data for:  Here Comes the Groom
Have 99 months of pageview data
Getting pageview data for:  The High and the Mighty (film)
Have 99 months of pageview data
Getting pageview data for:  High Noon
Have 99 months of pageview data
Getting pageview data for:  Hitler Lives
Have 99 months of pageview data
Getting pageview data for:  A Hole in the Head
Have 99 months of pageview data
Getting pageview data

Have 99 months of pageview data
Getting pageview data for:  Monsieur Vincent
Have 99 months of pageview data
Getting pageview data for:  Moonbird
Have 99 months of pageview data
Getting pageview data for:  The More the Merrier
Have 99 months of pageview data
Getting pageview data for:  Morning Glory (1933 film)
Have 99 months of pageview data
Getting pageview data for:  Moscow Strikes Back
Have 99 months of pageview data
Getting pageview data for:  Mother Wore Tights
Have 99 months of pageview data
Getting pageview data for:  Moulin Rouge (1952 film)
Have 99 months of pageview data
Getting pageview data for:  Mouse Trouble
Have 99 months of pageview data
Getting pageview data for:  Mr. Deeds Goes to Town
Have 99 months of pageview data
Getting pageview data for:  Mr. Smith Goes to Washington
Have 99 months of pageview data
Getting pageview data for:  Mrs. Miniver
Have 99 months of pageview data
Getting pageview data for:  Munro (film)
Have 99 months of pageview data
Getting pageview da

Have 99 months of pageview data
Getting pageview data for:  The Seventh Veil
Have 99 months of pageview data
Getting pageview data for:  Shane (film)
Have 99 months of pageview data
Getting pageview data for:  Shanghai Express (film)
Have 99 months of pageview data
Getting pageview data for:  She Wore a Yellow Ribbon
Have 99 months of pageview data
Getting pageview data for:  Shoeshine (film)
Have 99 months of pageview data
Getting pageview data for:  The Silent World
Have 99 months of pageview data
Getting pageview data for:  The Sin of Madelon Claudet
Have 99 months of pageview data
Getting pageview data for:  Since You Went Away
Have 99 months of pageview data
Getting pageview data for:  Skippy (film)
Have 99 months of pageview data
Getting pageview data for:  Sky Above and Mud Beneath
Have 99 months of pageview data
Getting pageview data for:  The Snake Pit
Have 99 months of pageview data
Getting pageview data for:  Snow White and the Seven Dwarfs (1937 film)
Have 99 months of page

Have 99 months of pageview data
Getting pageview data for:  Van Gogh (1948 film)
Have 99 months of pageview data
Getting pageview data for:  The Vanishing Prairie
Have 99 months of pageview data
Getting pageview data for:  The Virgin Spring
Have 99 months of pageview data
Getting pageview data for:  Viva Villa!
Have 99 months of pageview data
Getting pageview data for:  Viva Zapata!
Have 99 months of pageview data
Getting pageview data for:  Waikiki Wedding
Have 99 months of pageview data
Getting pageview data for:  The Walls of Malapaga
Have 99 months of pageview data
Getting pageview data for:  The War of the Worlds (1953 film)
Have 99 months of pageview data
Getting pageview data for:  Watch on the Rhine
Have 99 months of pageview data
Getting pageview data for:  Water Birds
Have 99 months of pageview data
Getting pageview data for:  The Way of All Flesh (1927 film)
Have 97 months of pageview data
Getting pageview data for:  West Side Story (1961 film)
Have 62 months of pageview dat